# PSPNetの推論

## 目標
セマンティックセグメンテーションの推論を実装できるようになる

## Library

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch

In [2]:
from utils.dataloader import make_datapath_list, DataTransform

# ファイルパスリストの作成
rootpath = './data/VOCdevkit/VOC2012/'
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath=rootpath)


## ネットワークの準備

In [ ]:
from utils.pspnet import PSPNet

# modelの外側を作って学習させた重みをロードする
net = PSPNet(n_classes=21)
state_dict = torch.load('./weights/pspnet50_30.pth', map_location={'cuda':'cpu'})
net.load_state_dict(state_dict)

print('ネットワーク設定完了：学習済みの重みをロードしました')

## 推論

In [ ]:
# 元画像表示
image_file_path = './data/cowboy-757575_640.jpg'
img = Image.open(image_file_path)  # height, width, channel
img_width, img_height = img.size
plt.imshow(img)
plt.show()

# 前処理
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)
transform = DataTransform(input_size=475, color_mean=color_mean, color_std=color_std)

# アノテーション画像からカラーパレットの情報を抜き出す
anno_file_path = val_anno_list[0]
anno_class_img = Image.open(anno_file_path)  # height, width
p_palette = anno_class_img.getpalettt()
phase = 'val'
img, anno_class_img = transform(phase, img, anno_class_img)

# 推論
net.eval()  # 推論モードへ
x = img.unsqueeze(0)  # ミニバッチ化
outputs = net(x)
y = outputs[0]  # AuxLoss側（y[1]）は無視

# 出力から最大クラスを求め、カラーパレット形式にして画像サイズをもとに戻す
y = y[0].detach().numpy()
y = np.argmax(y, axis=0)
anno_class_img = Image.fromarray(np.uint8(y), mode='P')
anno_class_img = anno_class_img.resize((img_width, img_height), ImageNEAREST)
anno_class_img.putpalette(p_palette)
plt.imshow(anno_class_img)
plt.show()

# 画像を透過させて重ねる
trans_img = Image.new('RGBA', anno_class_img.size, (0,0,0,0))
anno_class_img = anno_class_img.convert('RGBA')  # カラーパレット形式をRGBA形式に変換

for x in range(img_width):
    for y in range(img_height):
        pixel = anno_class_img.getpixel((x, y))
        r, g, b, a = pixel
        
        # (0,0,0)の背景ならそのままにして透過させる
        if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
            continue
        else:
            # それ以外の場合は色を用意した画像にピクセルを書き込む
            trans_img.putpixel((x, y), (r, g, b, 150))　　# 150 透過度
            
img = Image.open(image_file_path)
result = Image.alpha_composite(img.convert('RGBA'), trans_img)
plt.imshow(result)
plt.show()
            